In [9]:
# LabelEncoder 변환 값 원복 작업
df_train_y = pd.read_csv("https://raw.githubusercontent.com/eyejjang/BigData/main/dataset_yemoon/datasets/Part5/404_y_train.csv")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train_y['Segmentation'] = le.fit_transform(df_train_y['Segmentation'])
print(le.classes_)

['A' 'B' 'C' 'D']


In [ ]:
#import sklearn.preprocessing
#print(dir(sklearn.preprocessing))
#print(help(sklearn))
print(le.classes_)

################################################################################
## 데이터 파일 읽기
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata
import pandas as pd
df_train_x = pd.read_csv("/content/yemoonsaBigdata/datasets/Part5/404_x_train.csv")
df_train_y = pd.read_csv("/content/yemoonsaBigdata/datasets/Part5/404_y_train.csv")
df_test_x = pd.read_csv("/content/yemoonsaBigdata/datasets/Part5/404_x_test.csv")
################################################################################
## 데이터 확인
print("## 데이터 확인")
print(df_train_x.info())
print(df_train_x.describe(include='all'))
print(df_train_x.head())
print('*'*100)
print(df_test_x.info())
print(df_test_x.describe(include='all'))
print(df_test_x.head())
print('*'*100)
################################################################################
## 결측치 확인
print("## 결측치 확인")
print(df_train_x.isna().sum())
print(df_test_x.isna().sum())
################################################################################
## 결측치 처리
################################################################################
## 컬럼 그룹화
col_del = ['ID']
col_str = ['Gender','Ever_Married','Graduated','Profession','Spending_Score']
col_num = ['Age','Work_Experience','Family_Size']
#y = Segmentation
################################################################################
## 라벨인코팅
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in col_str :
    df_train_x[i] = le.fit_transform(df_train_x[i])
    df_test_x[i] = le.transform(df_test_x[i])

df_train_y['Segmentation'] = le.fit_transform(df_train_y['Segmentation'])

## 원 핫 인코딩
#df_train_x = pd.get_dummies(df_train_x)
#df_test_x = pd.get_dummies(df_test_x)
################################################################################
## 스케일링
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
df_train_x[col_num] = mm.fit_transform(df_train_x[col_num])
df_test_x[col_num] = mm.transform(df_test_x[col_num])

print("## 스케일링 후 확인")
print(df_train_x.head())
print(df_test_x.head())
################################################################################
## 트레이닝셋, 검증셋 분리
from sklearn.model_selection import train_test_split
train_x, valdn_x, train_y, valdn_y = train_test_split(df_train_x[col_str+col_num], df_train_y['Segmentation'], test_size = 0.2, stratify = df_train_y['Segmentation'])

print("## 데이터셋 분리 후 확인")
print(train_x.shape)
print(valdn_x.shape)
print(train_y.shape)
print(valdn_y.shape)
################################################################################
## 랜덤포레스트 모델링(회귀)
from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor(random_state=123)
RFR.fit(train_x, train_y)

## 랜덤포레스트 모델링(분류)
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=123)
RFC.fit(train_x, train_y)

## XG부스트 모델링(회귀)
from xgboost import XGBRegressor
XGBR = XGBRegressor(random_state=123)
XGBR.fit(train_x, train_y)

## XG부스트 모델링(분류)
from xgboost import XGBClassifier
XGBC = XGBClassifier(random_state=123)
XGBC.fit(train_x, train_y)
################################################################################
## 결과값 예측 (분류)
print("## 결과값 예측")
pred_RFC = RFC.predict(valdn_x)
pred_XGBC = XGBC.predict(valdn_x)

## F1 스코어(분류)
from sklearn.metrics import f1_score
score_RFC = f1_score(valdn_y, pred_RFC, average='macro')
score_XGBC = f1_score(valdn_y, pred_XGBC, average='macro')

## Accuracy(분류)
from sklearn.metrics import accuracy_score
score_RFC = accuracy_score(valdn_y, pred_RFC, average='macro')
score_XGBC = accuracy_score(valdn_y, pred_XGBC, average='macro')

## Recall(분류)
from sklearn.metrics import recall_score
score_RFC = recall_score(valdn_y, pred_RFC, average='macro')
score_XGBC = recall_score(valdn_y, pred_XGBC, average='macro')

## Precision(분류)
from sklearn.metrics import precision_score
score_RFC = precision_score(valdn_y, pred_RFC, average='macro')
score_XGBC = precision_score(valdn_y, pred_XGBC, average='macro')

print('score_RFC: ', score_RFC)
print('score_XGBC: ', score_XGBC)
################################################################################
## 결과값 예측 (분류-확률)
print("## 결과값 예측")
pred_RFC = RFC.predict_proba(valdn_x)[:,1]
pred_XGBC = XGBC.predict_proba(valdn_x)[:,1]

## ROC_AUC 확인
from sklearn.metrics import roc_auc_score
score_RFC = roc_auc_score(valdn_y, pred_RFC)
print('ROC_AUC_RFC_valdn: ', score_RFC)
score_XGBC = roc_auc_score(valdn_y, pred_XGBC)
print('ROC_AUC_XGBC_valdn: ', score_XGBC)
################################################################################
## 결과값 예측 (회귀)
print("## 결과값 예측")
pred_RFR = RFR.predict(valdn_x)
pred_XGBR = XGBR.predict(valdn_x)

## MAE (회귀)
from sklearn.metrics import mean_absolute_error
mae_RFR = mean_absolute_error(valdn_y, pred_RFR)
mae_XGBR = mean_absolute_error(valdn_y, pred_XGBR)
print('RMSE_RFR:', mae_RFR)
print('RMSE_XGBR:', mae_XGBR)

## MSE (회귀)
from sklearn.metrics import mean_squared_error
mse_RFR = mean_squared_error(valdn_y, pred_RFR)
mse_XGBR = mean_squared_error(valdn_y, pred_XGBR)
print('RMSE_RFR:', mse_RFR)
print('RMSE_XGBR:', mse_XGBR)

## RMSE (회귀)
from sklearn.metrics import mean_squared_error
import numpy as np
rmse_RFR = np.sqrt(mean_squared_error(valdn_y, pred_RFR))
rmse_XGBR = np.sqrt(mean_squared_error(valdn_y, pred_XGBR))
print('RMSE_RFR:', rmse_RFR)
print('RMSE_XGBR:', rmse_XGBR)
################################################################################
## 결과 데이터프레임 생성
pred_RFC = RFC.predict(df_test_x[col_str+col_num])
y_test_RFC = le.inverse_transform(pred_RFC)
result = pd.DataFrame({'ID':df_test_x['ID'], 'Segmentation':y_test_RFC})
print(result)
################################################################################
## 결과값 파일 저장
result.to_csv('04_02.csv', index=False)
################################################################################